# Ben Needs a Friend - LLM Agent for event listings
This is part of the "Ben Needs a Friend" tutorial. See all the notebooks and materials [here](https://github.com/bpben/ben_friend_25). Follow setup instructions there to use this notebook.

In this notebook, we set up a simple workflow for an agent to suggest some cool events from the [Boston Calendar](https://www.thebostoncalendar.com/).  

Since this requires a larger model than we've used in other notebooks, it will not work on Codespaces unless you switch to using an LLM provider like OpenAI.  

If you are using OpenAI:
- This has been tested with GPT-4o 
- You will need to set an environment variable `OPENAI_API_KEY` with your API key

In [ ]:
from pathlib import Path
from datetime import datetime
from bs4 import BeautifulSoup
import requests
from random import sample
from llamabot import  AgentBot, SimpleBot, tool


agent_model = "qwen2.5:7b"
openai_model = 'gpt-4o' # llamabot requires a model with structured output features
friend_model = "llama3.1:8b"

friend_prompt = """Your name is Friend. \
You are having a conversation with your close friend Ben. \
You and Ben are sarcastic and poke fun at one another. \
But you care about each other and support one another."""

### Using AgentBot
Our first example uses llamabot's `AgentBot`.  This is pretty experimental, but I think it displays some components of a simple bot.

In [ ]:
# Create the bot
bot = AgentBot(
    system_prompt=friend_prompt,
    functions=[],
    model_name=f"ollama_chat/{agent_model}",
)

In [ ]:
response = bot("What day is it today?")
response.content

This will almost always be wrong.  Why? Because our friend doesn't have access to a calendar! Let's fix that by defining a `tool`

Note - this will not always work, even with an 8 billion parameter model.  More on that in the slides.

In [ ]:
@tool
def today(text: str) -> str:
    """Returns today's date, use this when you need to get today's date.\
    The input should always be an empty string."""
    return datetime.today().strftime('%Y-%m-%d')

In [ ]:
# look at what is being provided to the model
print(today.json_schema)

In [ ]:
# Create the bot
bot = AgentBot(
    system_prompt=friend_prompt,
    functions=[today,],
    # often this will not work and we will need to use openai
    # model_name=f"ollama_chat/{agent_model}"
    model_name='gpt-4o'
)

In [ ]:
bot.tools

In [ ]:
# system prompt has been extended with additional information
bot.decision_bot.system_prompt.content

In [ ]:
response = bot("What day is it today?")

In [ ]:
response.content

It usually takes a few tries, but this typically work.

### Finding local events
A good friend will invite you to cool local events.  So why don't we give our AI friend access to that capability?

Our implementation here will scrape [The Boston Calendar](https://www.thebostoncalendar.com) and return a random assortment of events from there.

The result of the agent workflow should be telling us about all these cool events.  

This will often fail with a smaller model, we'll likely need to switch to using OpenAI's model to accomplish this.

In [ ]:
def parse_for_calendar(text: str) -> str:
    """
    Utility for converting text input to boston calendar URL
    """
    if len(text) == 1:
        # will assume today's month if we're looking at weekend number
        today = datetime.now()
        month = today.month
        day = today.day
        year = today.year
        url = f'https://www.thebostoncalendar.com/events?day={day}&month={month}&weekend={text}&year={year}'
    # this will fail if the string provided is not a date
    else:
        try:
            f_date = datetime.strptime(text, '%Y-%m-%d')
            day_of_month = f_date.day
            month = f_date.month
            year = f_date.year
            url = f'https://www.thebostoncalendar.com/events?day={day_of_month}&month={month}&year={year}'
        except ValueError:
            return 
    return url

def weekend(text: str) -> str:
    """Returns the single-digit weekend number for this weekend, \
    use this for any questions related to the weekend date. \
    The input should always be an empty string, \
    and this function will always return the weekend number."""
    today = datetime.now()
    
    # Calculate the weekend number within the month
    first_day_of_month = today.replace(day=1)
    weekend_number_within_month = (today - first_day_of_month).days // 7 + 1

    return weekend_number_within_month

@tool
def get_events(text: str) -> str:
    """Returns information about local events. \ 
    The input is either a date string in the format YYYY-MM-DD, \
    or it is a single-digit weekend number.\
    This function will return a list where \
    each element contains an event name, date and location as a tuple.\
    This function should be used to provide complete information about events."""
    # use the parsing utility to get a formatted url
    url = parse_for_calendar(text)
    if url is None:
        # give the LLM a useful response
        return f'Input "{text}" is not in the right format - it needs to be a date string or a weekend number'
    response = requests.get(url)     
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract data
    events = soup.find_all('div', class_='info')

    all_events = []
    # gather the first 3 events
    select_events = []
    # there's now a lot of constant pinned events
    for i, event in enumerate(events[10:]):
        if i > 2:
            break
        title = event.find('h3').text.strip()
        date = event.find('p', class_='time').text.strip()
        location = event.find('p', class_='location').text.strip()
        select_events.append((title, date, location))
    
    return select_events


In [ ]:
friend_bot = AgentBot(
    system_prompt=friend_prompt,
    functions=[get_events, today ],
    model_name=f"ollama_chat/{agent_model}",
    #model_name='gpt-4o'
)

In [ ]:
response = friend_bot("What events are happening today?")

In [ ]:
print(response.content)

In [ ]:
friend_bot.memory

If we use a large model, this will usually produce a useful response, even with the proper "friend" style.  But if you use the normal system prompt, sometimes you get more information.  

What if we split up this task? We'll depend on OpenAI to retrieve the information, but our smaller model to "style" it.  This what we might consider a "multi-agent" system.  

With llamabot, this is not implemented, but we can do a hack where we just wrap a simple bot in a function.

In [ ]:
# creating a styling prompt
styling_prompt = """Your name is Friend. \
You are having a conversation with your close friend Ben. \
You and Ben are sarcastic and poke fun at one another. \
But you care about each other and support one another. 

You know the following information:
{events_output}"""

@tool
def style_response(event_info: list, user_prompt: str) -> str:
    """
    Utility for applying "Friend" style to the event information. \
    Use this after gathering event information.
    """
    # Convert the list of tuples to a string
    event_info_str = "\n".join([f"Event: {event[0]}, Date: {event[1]}, Location: {event[2]}" for event in event_info])

    filled_styling_prompt = styling_prompt.format(events_output=event_info_str)
    
    styler_agent = SimpleBot(
        system_prompt=filled_styling_prompt,
      model_name=f"ollama_chat/{agent_model}",
    )

    return styler_agent(user_prompt)

In [ ]:
response = style_response([('PhotoWalks Beacon Hill Tour', 'Wednesday, May 07, 2025 1:00p', 'Beacon Hill'), ('Film Screening: Exit Through the Gift Shop', 'Wednesday, May 07, 2025 2:00p', 'Harvard Art Museums'), ('Girls Can Be Engineers: An In-person Book Reading with Author and Engineer Jamila H. Lindo at Discovery Museum', 'Wednesday, May 07, 2025 3:00p', 'Discovery Museum')],
               "What events are happening today?")

In [ ]:
multi_agent_bot = AgentBot(
    system_prompt=friend_prompt,
    functions=[get_events, today, style_response],
    model_name='gpt-4o'
)

In [ ]:
response = multi_agent_bot("What events are happening today?")

In [ ]:
print(response.content)